In [ ]:
import os, imaplib, email
from email.header import decode_header
from email import message_from_bytes
from datetime import datetime
from dotenv import load_dotenv
load_dotenv()

IMAP_SERVER = 'imap.gmail.com'
IMAP_PORT = 993  
USERNAME = os.getenv("user")
PASSWORD = os.getenv("pass")

In [88]:
mail = imaplib.IMAP4_SSL(IMAP_SERVER, IMAP_PORT)
mail.login(USERNAME, PASSWORD)

# Select the mailbox
mail.select('INBOX')

('OK', [b'6847'])

In [89]:
status, email_ids = mail.search(None, f'(SINCE {'06-Aug-2025'})')
email_id_list = email_ids[0].split()

In [90]:
status, email_ids = mail.search(None, 'ALL')
email_id_list = email_ids[0].split()

# Get the raw message for the latest email (for example)
latest_email_id = email_id_list[-1]

In [ ]:
#https://docs.python.org/3/library/email.header.html

# Fetch the email by ID
status, msg_data = mail.fetch(latest_email_id, "(RFC822)")

# Parse the email content
raw_email = msg_data[0][1]
msg = email.message_from_bytes(raw_email, policy=email.policy.default)

# Decode email subject
subject, _ = decode_header(msg["Subject"])[0]

if isinstance(subject, bytes):
    subject = subject.decode("utf-8")

# From
from_ = msg.get("From")

# Print basic headers
print(f"From: {from_}")
print(f"Subject: {subject}")

# Get the email body
if msg.is_multipart():
    for part in msg.walk():
        content_type = part.get_content_type()
        content_disposition = str(part.get("Content-Disposition"))

        if "attachment" not in content_disposition and content_type == "text/plain":
            body = part.get_payload(decode=True).decode()
            print("Body:", body)
            break
else:
    # Non-multipart email
    body = msg.get_payload(decode=True)
    print("Body -- 1:", type(body))
    body=body.decode()
    print("Body -- 2:", body)

From: Sathwik Methari <sathwikmethari@alumni.iitm.ac.in>
Subject: Test
Body: Here is an attached



In [92]:
# Logout
#mail.logout()

for attachment in msg.iter_attachments():
    # Process each attachment here
    filename = attachment.get_filename()
    print(filename)

1000001486.jpg


In [93]:
import sys
size_in_bytes=sys.getsizeof(body)

size_in_mb=size_in_bytes / (1024 * 1024)
print(f"Size in MB: {size_in_mb}")

Size in MB: 5.91278076171875e-05
